In [ ]:
#------------------------
# Google Colab上でのみ実行
#------------------------
import time
!git clone https://github.com/konnitiha3/MOD2NN.git

import sys
sys.path.append('/content/MOD2NN')

from google.colab import drive
drive.mount('/content/drive')

In [1]:
import time
import os
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import sys
import pandas as pd
from cxlayers import ImageToElectricField, CxMO, AngularSpectrum, CxD2NNIntensity, D2NNMNISTDetector, ImageResizing, ImageBinarization, D2NNMNISTFilter, CxD2NNFaradayRotation, Polarizer

tf.random.set_seed(1)
tf.experimental.numpy.experimental_enable_numpy_behavior()

print("TensorFlow:", tf.__version__)
print("Python:", sys.version)

plt.rcParams['font.size'] = 18

TensorFlow: 2.8.0
Python: 3.8.11 (default, Aug 16 2021, 12:04:33) 
[Clang 12.0.0 ]


In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0

In [8]:
class MOD2NN(tf.keras.models.Model):
  def __init__(self, **kwargs):
    super().__init__(**kwargs)
    # 複数モデルを入れ子にすることもOK
    self.model = self.create_model()
    # トラッカーを用意する（訓練、テスト共通で良い）
    self.loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    self.loss_tracker = tf.keras.metrics.SparseCategoricalCrossentropy()
    self.accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="accuracy")
    self.accuracy_tracker = tf.keras.metrics.SparseCategoricalAccuracy()

  def create_model(self):
    shape = (40, 40)
    inputs = tf.keras.layers.Input((28, 28))
    x = ImageResizing(shape)(inputs)
    x = ImageBinarization(0.5, 0, 1)(x)
    x = ImageToElectricField(shape)(x)
    x = CxMO(shape)(x)
    x = AngularSpectrum(shape, 531e-9, 0.5e-3, d=2.0e-6, n=1.0, method='expand')(x)
    x = CxD2NNIntensity(shape)(x)
    x = D2NNMNISTDetector(10)(x)
    return tf.keras.models.Model(inputs, x)

  # なくてもエラーは出ないが、訓練・テスト間、エポックの切り替わりで
  # トラッカーがリセットされないため、必ずmetricsのプロパティをオーバーライドすること
  # self.reset_metrics()はこのプロパティを参照している
  @property
  def metrics(self):
    return [self.loss_tracker, self.accuracy]

  def train_step(self, data):
    x, y = data

    with tf.GradientTape() as tape:
      pred = self.model(x)
      loss_value = self.loss_object(y, pred)
    # 全体（self）に対する偏微分か、特定モデル（self.model）に対する微分かは場合により変わる
    # このケースではどちらでも同じだが、GANでは使い分ける必要がある
    grads = tape.gradient(loss_value, self.trainable_weights)
    self.optimizer.apply_gradients(zip(grads, self.trainable_weights))

    # エポックの切り替わりのトラッカーのリセットは、self.reset_metrics()で自動的に行われる
    self.loss_tracker.update_state(y, pred)
    self.accuracy_tracker.update_state(y, pred)

    return {
      "loss": self.loss_tracker.result(),
      "accuracy": self.accuracy_tracker.result()
    }

  def test_step(self, data):
    x, y = data

    pred = self.model(x)
    loss = self.loss_object(y, pred)

    # 訓練・テストの切り替わりのトラッカーのリセットは、self.reset_metrics()で自動的に行われる
    self.loss_tracker.update_state(loss)
    return {
      "loss": self.loss_tracker.result()
    }

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 28, 28)]          0         
                                                                 
 image_resizing_1 (ImageResi  (None, 40, 40)           0         
 zing)                                                           
                                                                 
 image_binarization_1 (Image  (None, 40, 40)           0         
 Binarization)                                                   
                                                                 
 image_to_electric_field_1 (  (None, 2, 2, 40, 40)     0         
 ImageToElectricField)                                           
                                                                 
 cx_mo_2 (CxMO)              (None, 2, 2, 40, 40)      1600      
                                                           

In [9]:
model =MOD2NN()
model.compile(optimizer=tf.keras.optimizers.Adam())
model.fit(x_train, y_train, batch_size=64, epochs=10)

2022-04-03 18:52:45.701755: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-04-03 18:52:50.953599: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-04-03 18:53:53.696737: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Loss test: 2.3027076721191406
